In [1]:
#<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>#
#                          CLASSES                           #
#<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>#

import random
import time
from IPython.display import clear_output

#<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>#

class Population:
    """
    Class storing key information about the population space, including fitness and 
    exclusion functions, data conversions, and both crossover and mutation details. 
    Initializes with a list of Individual objects.
    
    Note that all functions stored as class variables should be defined outside of 
    the class, and assigned with their setters.

    Input:
        - initial_list : list of objects of class'Individual'

    """

#=======================[CONSTRUCTOR]========================#

    def __init__(self, initial_list):
        
        #Functions
        self.fitness_function   = None
        self.exclusion_function = None
        self.toList             = None
        self.fromList           = None
        
        #Data
        self.initial_population = initial_list
        self.current_population = self.initial_population
        self.selection          = []
        self._elite_element     = None
        self.generation_count   = 0
        
        #Parameters
        self._mutationspace     = None
        self._mutationchance    = None
        self._crossoverchance   = None
        self.display            = False
        self.run_time           = None
        self.start_time         = time.time()
        
#=========================[METHODS]==========================#

#-------------------------[GETTERS]--------------------------#

    #Data
    def getSelection(self):
        return self.selection
    
    def getInitialPopulation(self):
        return self.initial_population
    
    def getCurrentPopulation(self):
        return self.current_population
    
    def getEliteElement(self):
        return self._elite_element
    
    def getCount(self):
        return self.generation_count
    
    #Parameters
    def getMutationSpace(self):
        return self._mutationspace
    
    def getMutationChance(self):
        return self._mutationchance
    
    def getCrossoverChance(self):
        return self._crossoverchance
    
    def getRuntime(self):
        return self.run_time
    
#-------------------------[SETTERS]--------------------------#
    
    #Functions
    def setFitnessFunction(self, func):
        self.fitness_function = func
        
    def setExclusionFunction(self, func):
        self.exclusion_function = func
        
    def setToListFunction(self, func):
        self.toList = func
        
    def setFromListFunction(self, func):
        self.fromList = func
    
    #Data
    def setCurrentPopulation(self, new_population):
        self.current_population = new_population
        
    def setEliteElement(self, elite_element):
        self._elite_element = elite_element
        
    def setDisplay(self, flag):
        self.display = flag
        
    #Parameters
    def setMutationSpace(self, number_list):
        self._mutationspace = number_list
    
    def setMutationChance(self, value):
        self._mutationchance = value
    
    def setCrossoverChance(self, value):
        self._crossoverchance = value
        
    def setRuntime(self, minutes):
        self.run_time = minutes * 60
        
#------------------------[INTERNAL]--------------------------#

    def clearSelection(self):
        self.selection = []
        
    def shuffleSelection(self):
        random.shuffle(self.selection)
        
    def sortPopulation(self):
        self.current_population.sort(key = lambda x : x.getGrade())
        
    def nextGeneration(self):
        self.generation_count += 1
        
    def crossoverFlag(self):
        """
        Returns True with probability equal to  the stored Population._crossover_chance, and False otherwise
        
        """
        crossover_probability  = [self.getCrossoverChance(), 1 - self.getCrossoverChance()]
        crossover_distribution = GeneralDiscreteDistribution(crossover_probability)
        
        if crossover_distribution.get_random_element() == 0:
            
            return True
        
        else:
            
            return False
    
#-----------------[STORED FUNCTION CALLS]--------------------#

    def toList(self, data):
        return self.toList(data)
    
    def fromList(self, data):
        return self.fromList(data)
    
    def grade(self, list_data):
        return self.fitness_function(self.fromList(list_data))
    
    def exclude(self, list_data):
        return self.exclusion_function(self.fromList(list_data))
    
#------------------------[DISPLAY]---------------------------#

    def displayProgress(self, message, current_operations, total_operations):
        
        clear_output(wait=True)
        timestamp = time.strftime("%H:%M:%S", time.gmtime(time.time() - self.start_time))
        print("[" + timestamp +"]", message, str(current_operations) + "/" +  str(total_operations))
        if self.generation_count > 0:
            print("Current Elite:", self.getEliteElement().getGrade())

#---------------------[ELITE ELEMENTS]-----------------------#
    
    
    def findEliteElement(self):
        """
        Sorts Population.current_population by fitness score, and returns a copy of the first 
        Individual that returns False in Population.exclude()

        Output:
            - object of class 'Individual'
        """
        
        self.sortPopulation()
        
        for individual in self.getCurrentPopulation():
            
            if not self.exclude(self.fromList(individual.getList())):
                
                if not individual.getGrade() < 0:
                    
                    if not individual.getGrade() == -infinity:
        
                        return self.getCurrentPopulation()[0].copy()
    
    def checkEliteElement(self):
        """
        Calls Population.findEliteElement(), and if the found element has better fitness score 
        than the stored elite element, replaces the stored elite element. Otherwise, removes 
        the element of worst fitness score from the current population and inserts stored elite 
        element to ensure population fitness does not rise.
        
        """
        
        eliteElement = self.findEliteElement()
        
        if eliteElement.getGrade() <= self.getEliteElement().getGrade():
            
            self.setEliteElement(eliteElement)
            
        else:
            
            self.current_population.pop(len(self.getCurrentPopulation())-1)
            self.current_population.insert(0, self.getEliteElement().copy())
    
#-------------------[GENETIC FUNCTIONS]----------------------#
        
    def gradePopulation(self):
        """
        Loops through Popoulation.current_population and assigns each Individual the fitness score 
        returned by self.fitness_function

        """
        for i in range(len(self.getCurrentPopulation())):
            
            if self.display:
                
                message = "Grading generation " + str(self.generation_count) + "..."
                self.displayProgress(message, i+1, len(self.getCurrentPopulation()))
            
            individual = self.getCurrentPopulation()[i]
            individual.setGrade(self.grade(individual.getList()))
            
            
            
            
    def selectCandidates(self):
        """
        Generates a probability distribution based on the fitness scores of the current population, 
        and then selects Individuals from the current population with probability corresponding to 
        their fitness score. Stores the selected elements in Population.selection
        
        """
        self.clearSelection()
        
        selection_probabilities = [exp(-1*individual.getGrade()) for individual in self.getCurrentPopulation()]
        selection_distribution  = GeneralDiscreteDistribution(selection_probabilities) 
        
        while len(self.getSelection()) < len(self.getInitialPopulation()):
            
            if self.display:
                
                message = "Selecting candidates..."
                self.displayProgress(message, len(self.getSelection()), len(self.getInitialPopulation()))
            
            selection_index = selection_distribution.get_random_element()
            
            self.selection.append(self.current_population[selection_index])

            
    def populationCrossover(self):
        """
        Shuffles the current population, pairs parents and calls crossoverFlag() for each pair. If True, 
        then calls Individual.crossover(), and if both children return False on exclusion function, 
        appends them to the new population list. If either child returns True on exclusion function, 
        or if crossoverFlag returns False, then parents are appended to the new population list.
        
        """
        self.shuffleSelection()
        
        half_popsize         = len(self.getCurrentPopulation())/2
        crossover_population = []
        
        for i in range(half_popsize):
            
            if self.display:
                
                message = "Performing crossover..."
                self.displayProgress(message,i+1,half_popsize)
            
            parent_0 = self.getSelection()[i]
            parent_1 = self.getSelection()[i + half_popsize]
            
            if self.crossoverFlag():

                child_list = parent_0.crossover(parent_1)
                
                for j in range(len(child_list)):

                    if self.exclude(child_list[j]):
                        
                        crossover_population.append(parent_0) if i == 0 else crossover_population.append(parent_1)

                    else:
                        
                        crossover_population.append(Individual(child_list[j]))
                        
            else:
                
                crossover_population.append(parent_0)
                crossover_population.append(parent_1)
                
        self.setCurrentPopulation(crossover_population)
        
    def populationMutation(self):
        """
        Loops through the current population, makes a copy of each Individual and calls the 
        Individual.mutation() method. If the mutated copy returns False on the exclusion function, 
        replace the original individual with the mutated copy.
        
        """
        
        for i in range(len(self.getCurrentPopulation())):
            
            individual = self.getCurrentPopulation()[i]
            
            if self.display:
                
                message = "Mutating population..."
                self.displayProgress(message, i+1, len(self.getCurrentPopulation()))
            
            mutation = individual.copy()
            mutation.mutate(self.getMutationSpace(), self.getMutationChance())
            
            if not self.exclude(mutation.getList()):
                
                self.getCurrentPopulation()[i] = mutation

#<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>#


class Individual:
    """
    An object that stores data as a list for each element within a population, as well as its 
    associated fitness score. Contains methods to call on itself for each key stage in the 
    genetic algorithm.
    
    Input:
        - data_as_list : a list representing the data the individual should contain
    """
    
#=======================[CONSTRUCTOR]========================#
    
    def __init__(self, data_as_list):
        
        self._list           = data_as_list
        self._grade          = None
        self._data           = (self._list, self._grade)
        self.fitness_funtion = None
        
#=========================[METHODS]==========================#

#-------------------------[GETTERS]--------------------------#

    def getList(self):
        return self._list
    
    def getGrade(self):
        return self._grade
    
    def getData(self):
        return self._data
    
#-------------------------[SETTERS]--------------------------#

    def setList(self, new_list):
        self._list = new_list
        self.update()
        
    def setGrade(self, new_grade):
        self._grade = new_grade
        self.update()
    
#------------------------[INTERNAL]--------------------------#
    
    def update(self):
        self._data  = (self._list, self._grade)
        
#---------------------[IMPLEMENTATION]-----------------------#

    def copy(self):
        
        self_copy = Individual(self.getList())
        self_copy.setGrade(self.getGrade())
        
        return self_copy
    
    def crossover(self, crossover_partner):
        '''
        Takes a partner and picks a random crossover point, then returns the two lists formed by cutting 
        the list data for two Individuals at the chosen point and attatching the respective cuts.
        
        Input:
            - crossover_partner : object of class Individual
        Output:
            - List of two elements, each a list
        
        '''
        
        parent_a = self.getList()
        parent_b = crossover_partner.getList()
        cross_index = random.choice([_ for _ in range(1,len(self.getList()))])
        
        child_a = [parent_a[i] for i in range(cross_index)] + [parent_b[j] for j in range(cross_index, len(parent_a))]
        child_b = [parent_b[i] for i in range(cross_index)] + [parent_a[j] for j in range(cross_index, len(parent_a))]       
        
        return [child_a, child_b] 
    
    def mutate(self, probability_space, mutation_chance = 0.05):
        '''
        Forms a probability distribution to trigger mutation, then loops through the Individual's data list. 
        If mutation is triggered, replaces element in the data list with a random element chosen from the 
        probability space.
        
        Inputs:
            - probability_space : List of acceptable mutations
            - mutation_chance   : Probability (as a decimal) of triggering a mutation on any element in the data list
        Output:
            - List with two elements, each a list
        
        '''
        
        mutation_dist = GeneralDiscreteDistribution([mutation_chance, 1 - mutation_chance])
        
        for i in range(len(self.getList())):
            
            mutate_flag = mutation_dist.get_random_element()
            
            if mutate_flag == 0:
                
                self._list[i] = random.choice(probability_space)
                
        self.update()
        
        return self.getList()
    
    def child_space(self, *args):
        '''
        Takes a list of Individuals and produces all children formed by choosing random elements from the 
        Individuals' data lists.
        
        Inputs:
            - args : any number of objects of class Individual
        Output:
            - list with each element a list representing a unique child
        '''
        
        parent_list      = [self.getList()] + [arg.getList() for arg in args]
        child_list       = []
        comb_list        = []
        parent_index     = 0
        
        for parent in parent_list:
            
            for i in range(len(parent)):
            
                comb_list.append(parent_index)
                
            parent_index += 1
                
        combinations = Arrangements(comb_list, min([len(parent) for parent in parent_list]))
        
        for comb in combinations.list():
            
            child = []
            
            for i in range(len(comb)):
                
                child.append(parent_list[comb[i]][i])
                
            child_list.append(child)
        
        return child_list

#<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>#


In [ ]:
## Incorporate prime factorization of coefficients, and ideals generated by points with the Individual Class
## Want to check -- how often does prime p and prime ideal P appear in points of height smaller (or larger) than a given bound

load("DynamicalHeight.py")
load("initialization.py")

def initialize(**kwargs):
    
    n                = kwargs.pop("iterations")
    d                = kwargs.pop("degree")
    b                = kwargs.pop("constant")
    gen_size         = kwargs.pop("gen_size")
    display          = kwargs.pop("display")
    crossover_chance = kwargs.pop("crossover_chance")
    mutation_chance  = kwargs.pop("mutation_chance")
    mutation_space   = kwargs.pop("mutation_space")
    run_time         = kwargs.pop("run_time")
    galois           = kwargs.pop("galois")
    
    R.<x>            = PolynomialRing(ZZ)
    phi              = x^d + b
    phi_factors      = factor(nest(phi,n,x) - x)
    linear_factors   = [phi_factors[i][0]^phi_factors[i][1] for i in range(len(phi_factors))]
    
    if galois:
        K                = NumberField(linear_factors) 
        L.<a>            = K.galois_closure()
    else:
        L.<a>            = NumberField(linear_factors[len(linear_factors)-1])
    P.<X,Y>          = ProjectiveSpace(L,1)
    D                = DynamicalSystem([X^2 - Y^2, Y^2])
    population       = []
    
    def toList(point):
        
        point_as_list = []
        
        for i in range(L.degree()):
            
            point_as_list.append(point[i])
            
        return point_as_list
    
    def fromList(point_as_list):
        
        point = 0
        
        for i in range(L.degree()):
            
            point += point_as_list[i]*L.gen()^i
            
        return point
    
    def exclude(point):
        
        if P(point).is_preperiodic(D):
            
            return True
        
        else:
            
            fitnessScore = fitnessFunction(point)
        
            if fitnessScore == -infinity:
            
                return True
        
            elif fitnessScore < 0.002:
            
                return True
                
            else:
            
                return False
    
    def fitnessFunction(point):
        
        #return D.canonical_height(P(point))
        
        f = R(point.denominator()^L.degree()*point.minpoly())
        
        return height(f, prec=5000)
    
    
    while len(population) < gen_size:
        
        if display:
        
            clear_output(wait=True)
            print("Generating initial population...", str(len(population) + 1) + "/" + str(gen_size))
        
        alpha = L.random_element()
        
        if not exclude(alpha):
            
            population.append(Individual(toList(alpha)))
            
    initial_population = Population(population)
    
    initial_population.setToListFunction(toList)
    initial_population.setFromListFunction(fromList)
    initial_population.setFitnessFunction(fitnessFunction)
    initial_population.setExclusionFunction(exclude)
    initial_population.setCrossoverChance(crossover_chance)
    initial_population.setMutationChance(mutation_chance)
    initial_population.setMutationSpace(mutation_space)
    initial_population.setRuntime(run_time)
    initial_population.setDisplay(display)
    

    return initial_population

In [2]:
#test case

load("DynamicalHeight.py")

kwargs = {
    "iterations"       : 3,
    "degree"           : 2,
    "constant"         : -1,
    "gen_size"         : 10, 
    "display"          : True,
    "crossover_chance" : 0.8,
    "mutation_chance"  : 0.1,
    "mutation_space"   : [i/100 for i in range(-1000,1001)],
    "run_time"         : 1,
    "galois"           : False
}

myPop = initialize(**kwargs)
myPop.gradePopulation()
myPop.setEliteElement(myPop.findEliteElement())
    
while time.time() - myPop.start_time < myPop.getRuntime():
    
    myPop.nextGeneration()
    myPop.selectCandidates()
    myPop.populationCrossover()
    myPop.populationMutation()
    myPop.gradePopulation()
    myPop.checkEliteElement()

AttributeError: 'tuple' object has no attribute 'gradePopulation'